Building Logic Gates and Adders Using Neural Networks From Scratch

In [1]:
import numpy as np

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_derivative(a):
    return a * (1 - a)


Neural network components: initialization, forward propagation, cost, backpropagation, and parameter updates.

In [2]:
def initialize(n_x, n_h, n_y):
    np.random.seed(1)
    W1 = np.random.randn(n_h, n_x)
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h)
    b2 = np.zeros((n_y, 1))
    return W1, b1, W2, b2


In [3]:
def forward_prop(X, W1, b1, W2, b2):
    Z1 = W1 @ X + b1
    A1 = sigmoid(Z1)
    Z2 = W2 @ A1 + b2
    A2 = sigmoid(Z2)
    return Z1, A1, Z2, A2


In [4]:
def compute_cost(A2, Y):
    m = Y.shape[1]
    cost = -(1/m) * np.sum(Y*np.log(A2) + (1-Y)*np.log(1-A2))
    return cost


In [5]:
def backward_prop(X, Y, Z1, A1, A2, W2):
    m = X.shape[1]

    dZ2 = A2 - Y
    dW2 = (1/m) * dZ2 @ A1.T
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)

    dZ1 = (W2.T @ dZ2) * sigmoid_derivative(A1)
    dW1 = (1/m) * dZ1 @ X.T
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)

    return dW1, db1, dW2, db2


In [6]:
def update(W1, b1, W2, b2, dW1, db1, dW2, db2, lr):
    W1 -= lr * dW1
    b1 -= lr * db1
    W2 -= lr * dW2
    b2 -= lr * db2
    return W1, b1, W2, b2


Neural Network Class

In [7]:
class NeuralNetwork:
    def __init__(self, n_x, n_h, n_y):
        self.W1, self.b1, self.W2, self.b2 = initialize(n_x, n_h, n_y)

    def train(self, X, Y, lr=0.5, epochs=10000):
        for _ in range(epochs):
            Z1, A1, Z2, A2 = forward_prop(X, self.W1, self.b1, self.W2, self.b2)
            dW1, db1, dW2, db2 = backward_prop(X, Y, Z1, A1, A2, self.W2)
            self.W1, self.b1, self.W2, self.b2 = update(
                self.W1, self.b1, self.W2, self.b2,
                dW1, db1, dW2, db2, lr
            )

    def predict(self, X):
        _, _, _, A2 = forward_prop(X, self.W1, self.b1, self.W2, self.b2)
        return (A2 > 0.5).astype(int)


Perceptron (single neuron) used to build AND gate and test XOR.

In [8]:
X_and = np.array([[0,0,1,1],
                  [0,1,0,1]])
Y_and = np.array([[0,0,0,1]])

and_nn = NeuralNetwork(2,1,1)
and_nn.train(X_and, Y_and, lr=1, epochs=5000)
print(and_nn.predict(X_and))


[[0 0 0 1]]


XOR Failure

In [9]:
X_xor = np.array([[0,0,1,1],
                  [0,1,0,1]])
Y_xor = np.array([[0,1,1,0]])

xor_fail = NeuralNetwork(2,1,1)
xor_fail.train(X_xor, Y_xor, lr=1, epochs=5000)
print(xor_fail.predict(X_xor))


[[0 1 0 0]]


XOR With Hidden Layer

In [10]:
xor_nn = NeuralNetwork(2,2,1)
xor_nn.train(X_xor, Y_xor, lr=0.5, epochs=10000)
print(xor_nn.predict(X_xor))


[[0 0 1 1]]


Full adder and ripple carry adder.

In [11]:
def XOR(a,b): return a ^ b
def AND(a,b): return a & b
def OR(a,b): return a | b

def full_adder(A,B,Cin):
    s1 = XOR(A,B)
    Sum = XOR(s1,Cin)
    c1 = AND(A,B)
    c2 = AND(s1,Cin)
    Cout = OR(c1,c2)
    return Sum, Cout


Ripple Carry Adder

In [12]:
def ripple_add(A,B):
    carry = 0
    result = []
    for i in range(len(A)-1,-1,-1):
        s, carry = full_adder(A[i],B[i],carry)
        result.insert(0,s)
    result.insert(0,carry)
    return result


In [13]:
A = [1,0,1,1]   # 11
B = [0,1,0,1]   # 5

print(ripple_add(A,B))


[1, 0, 0, 0, 0]
